# Class III - An introduction to PowerModels.jl

*Los Alamos National Laboratory Grid Science Winter School, 2019*

Welcome! This tutorial will introduce you to the basics of the [PowerModels.jl](https://github.com/lanl-ansi/PowerModels.jl) package. If you haven't yet, work through [Class I - An introduction  to Julia](Class%20I%20-%20An%20introduction%20to%20Julia.ipynb) and [Class II - An introduction  to JuMP](Class%20II%20-%20An%20introduction%20to%20JuMP.ipynb) first.

As in Class II, run the following magic sauce to check we're good to go.

In [25]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
println("Excellent! Everything is good to go!")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Excellent! Everything is good to go!


### Background 

This [presentation](https://github.com/lanl-ansi/tutorial-grid-science-2019/blob/master/assets/infrastructure_optimization_in_julia.pdf) provides motivation and context for this notebook.

Some additional informaiton is also available at,
* [PowerModels Documentation](https://lanl-ansi.github.io/PowerModels.jl/stable/)
* [PowerModels Network Model Update](https://youtu.be/j7r4onyiNRQ)
* [PSCC 2018 Presentation](https://youtu.be/AEEzt3IjLaM)
* [JuMP Dev 2017 Presentation](https://youtu.be/W4LOKR7B4ts)


### Working with the Network Model

A simple 5-bus network model is provided with this tutorial.  The matpower file can be viewed [here](../edit/data/pglib_opf_case5_pjm.m).

The `parse_file` function is used to load a text files into the PowerModels data model, 

In [26]:
using PowerModels

data = PowerModels.parse_file("data/pglib_opf_case5_pjm.m")

Dict{String,Any} with 13 entries:
  "bus"            => Dict{String,Any}("4"=>Dict{String,Any}("zone"=>1,"bus_i"=…
  "source_type"    => "matpower"
  "name"           => "pglib_opf_case5_pjm"
  "dcline"         => Dict{String,Any}()
  "source_version" => "2"
  "gen"            => Dict{String,Any}("4"=>Dict{String,Any}("pg"=>1.0,"model"=…
  "branch"         => Dict{String,Any}("4"=>Dict{String,Any}("br_r"=>0.00108,"r…
  "storage"        => Dict{String,Any}()
  "baseMVA"        => 100.0
  "per_unit"       => true
  "areas"          => Dict{String,Any}("1"=>Dict{String,Any}("col_1"=>1,"col_2"…
  "shunt"          => Dict{String,Any}()
  "load"           => Dict{String,Any}("1"=>Dict{String,Any}("load_bus"=>2,"sta…

In this case the file parser generated a few informational of messages.  

The first `info` message indicates that the file contains a data block named `area` that is not part of the Matpower data specification.  Still the parser will load this data using some standards for [Matpower data extentions](https://lanl-ansi.github.io/PowerModels.jl/stable/network-data/#Working-with-Matpower-Data-Files-1).

The second block of `warn` messages indicates that PowerModels has detected that the generator cost model could be simplified.  In this case a linear function specified as a quadratic function was transformed into a linear function.  PowerModels makes a best effort to clean data while parsing, but the best practice is to clean datasets until they do not produce any warning messages in PowerModels.

Let us use the silence command to ignore similar warnings for the rest of this tutorial.

In [27]:
PowerModels.silence()

Now, lets look at some of the network data.  The data for the first bus, load and branch can be viewed as follows,

In [28]:
data["bus"]["1"]

Dict{String,Any} with 10 entries:
  "zone"     => 1
  "bus_i"    => 1
  "bus_type" => 2
  "vmax"     => 1.1
  "area"     => 1
  "vmin"     => 0.9
  "index"    => 1
  "va"       => 0.0
  "vm"       => 1.0
  "base_kv"  => 230.0

In [29]:
data["load"]["1"]

Dict{String,Any} with 5 entries:
  "load_bus" => 2
  "status"   => 1
  "qd"       => 0.9861
  "pd"       => 3.0
  "index"    => 1

Notice that the "pd" value is 3.0, while in the Matpower file it was 300.  By default all data in InfrastructureModels is in per-unit.  The functions `make_mixed_units` and `make_per_unit` can be used for switching back and forth.

In [30]:
data["branch"]["1"]

Dict{String,Any} with 18 entries:
  "br_r"        => 0.00281
  "rate_a"      => 4.0
  "shift"       => 0.0
  "rate_b"      => 4.0
  "br_x"        => 0.0281
  "rate_c"      => 4.0
  "g_to"        => 0.0
  "g_fr"        => 0.0
  "b_fr"        => 0.00356
  "f_bus"       => 1
  "br_status"   => 1
  "t_bus"       => 2
  "b_to"        => 0.00356
  "index"       => 1
  "angmin"      => -0.523599
  "angmax"      => 0.523599
  "transformer" => false
  "tap"         => 1.0

That's great, but looking at components one-by-one can get boring fast.

All InfrastructureModels packages provide a `print_summary` function that prints a table-like summary of the network data to the terminal.

In [31]:
PowerModels.print_summary(data)

Metadata
  baseMVA: 100.000
  name: pglib_opf_case5_pjm
  per_unit: true
  source_type: matpower
  source_version: 2

Table Counts
  bus: 5
  load: 3
  shunt: 0
  gen: 5
  storage: 0
  branch: 6
  dcline: 0
  areas: 1


Table: bus
         bus_i, bus_type
      1:     1,        2
      2:     2,        1
      3:     3,        2
      4:     4,        3
      5:     5,        2

  default values:
    base_kv: 230.000
    vm: 1.000
    va: 0.000
    vmin: 0.900
    vmax: 1.100
    area: 1
    zone: 1


Table: load
         load_bus,    pd,    qd
      1:        2, 3.000, 0.986
      2:        3, 3.000, 0.986
      3:        4, 4.000, 1.315

  default values:
    status: 1


Table: gen
         gen_bus,    pg,  pmax,   qmin,  qmax
      1:       1, 0.200, 0.400, -0.300, 0.300
      2:       1, 0.850, 1.700, -1.275, 1.275
      3:       3, 2.600, 5.200, -3.900, 3.900
      4:       4, 1.000, 2.000, -1.500, 1.500
      5:       5, 3.000, 6.000, -4.500, 4.500

  default values:
    qg: 0.00

Dictionary comprehensions also provide an easy way to take slices of the data.

For example lets inspect the generation profile and output capabilities using comprehensions.

In [32]:
[gen["pg"] for (i,gen) in data["gen"]]

5-element Array{Float64,1}:
 1.0 
 0.2 
 3.0 
 0.85
 2.6 

In [33]:
[gen["pg"] + gen["qg"]im for (i,gen) in data["gen"]]

5-element Array{Complex{Float64},1}:
  1.0 + 0.0im
  0.2 + 0.0im
  3.0 + 0.0im
 0.85 + 0.0im
  2.6 + 0.0im

In [34]:
[(gen["qmin"], gen["qmax"]) for (i,gen) in data["gen"]]

5-element Array{Tuple{Float64,Float64},1}:
 (-1.5, 1.5)    
 (-0.3, 0.3)    
 (-4.5, 4.5)    
 (-1.275, 1.275)
 (-3.9, 3.9)    

### Solving Optimal Power Flow (OPF)

Before we can solve a power model, we need a solver.  Lets start with the simple linear programming solver GLPK and try to solve a DC OPF model.

In [35]:
using GLPK
lp_solver = with_optimizer(GLPK.Optimizer)

result = run_dc_opf(data, lp_solver)

┌ Warning: MathOptInterface.VariablePrimalStart() is not supported by MathOptInterface.Bridges.LazyBridgeOptimizer{GLPK.Optimizer,MathOptInterface.Utilities.UniversalFallback{MathOptInterface.Bridges.AllBridgedConstraints{Float64}}}. This 
│   information will be discarded. = information will be discarded.
└ @ MathOptInterface.Utilities /Users/carleton/.julia/packages/MathOptInterface/C3lip/src/Utilities/copy.jl:133


Dict{String,Any} with 10 entries:
  "solve_time"         => 9.39369e-5
  "optimizer"          => "GLPK"
  "termination_status" => OPTIMAL
  "dual_status"        => FEASIBLE_POINT
  "primal_status"      => FEASIBLE_POINT
  "objective"          => 17479.9
  "solution"           => Dict{String,Any}("baseMVA"=>100.0,"gen"=>Dict{String,…
  "machine"            => Dict("cpu"=>"Intel(R) Core(TM) i7-4870HQ CPU @ 2.50GH…
  "data"               => Dict{String,Any}("name"=>"pglib_opf_case5_pjm","bus_c…
  "objective_lb"       => -Inf

The result object contains a variety of useful information about the optimization problem solved, including the objective value and wall clock runtime.  Detailed documentation of the result dictionary is available [here](https://lanl-ansi.github.io/PowerModels.jl/stable/result-data/), however most interesting point is the `solution` data.

In [36]:
result["solution"]

Dict{String,Any} with 4 entries:
  "baseMVA"  => 100.0
  "gen"      => Dict{String,Any}("4"=>Dict{String,Any}("qg"=>NaN,"pg"=>0.0),"1"…
  "bus"      => Dict{String,Any}("4"=>Dict{String,Any}("va"=>0.0,"vm"=>1),"1"=>…
  "per_unit" => true

Not every useful, lets try `print_summary`,

In [37]:
PowerModels.print_summary(result["solution"])

Metadata
  baseMVA: 100.000
  per_unit: true

Table Counts
  bus: 5
  gen: 5


Table: bus
             va
      1:  0.057
      2: -0.014
      3: -0.008
      4:  0.000
      5:  0.072

  default values:
    vm: 1


Table: gen
            pg
      1: 0.400
      2: 1.700
      3: 3.235
      4: 0.000
      5: 4.665

  default values:
    qg: NaN0000


Looks great!  But `vm` and `qg` are missing...  On to AC OPF!

In [38]:
result = run_ac_opf(data, lp_solver)

┌ Warning: MathOptInterface.VariablePrimalStart() is not supported by MathOptInterface.Bridges.LazyBridgeOptimizer{GLPK.Optimizer,MathOptInterface.Utilities.UniversalFallback{MathOptInterface.Bridges.AllBridgedConstraints{Float64}}}. This 
│   information will be discarded. = information will be discarded.
└ @ MathOptInterface.Utilities /Users/carleton/.julia/packages/MathOptInterface/C3lip/src/Utilities/copy.jl:133


MathOptInterface.UnsupportedAttribute{MathOptInterface.NLPBlock}: MathOptInterface.UnsupportedAttribute{MathOptInterface.NLPBlock}: Attribute MathOptInterface.NLPBlock() is not supported by the model.

#### Quiz Question
That was ugly... What happened?!?


Lets try using an NLP solver.

In [39]:
using Ipopt
nlp_solver = with_optimizer(Ipopt.Optimizer, print_level=0)

result = run_ac_opf(data, nlp_solver)

Dict{String,Any} with 10 entries:
  "solve_time"         => 0.0222788
  "optimizer"          => "SolverName() attribute not implemented by the optimi…
  "termination_status" => LOCALLY_SOLVED
  "dual_status"        => FEASIBLE_POINT
  "primal_status"      => FEASIBLE_POINT
  "objective"          => 17551.9
  "solution"           => Dict{String,Any}("baseMVA"=>100.0,"gen"=>Dict{String,…
  "machine"            => Dict("cpu"=>"Intel(R) Core(TM) i7-4870HQ CPU @ 2.50GH…
  "data"               => Dict{String,Any}("name"=>"pglib_opf_case5_pjm","bus_c…
  "objective_lb"       => -Inf

It worked!  Lets look at the solution,

In [40]:
PowerModels.print_summary(result["solution"])

Metadata
  baseMVA: 100.000
  per_unit: true

Table Counts
  bus: 5
  gen: 5


Table: bus
            vm,     va
      1: 1.078,  0.049
      2: 1.084, -0.013
      3: 1.100, -0.010
      4: 1.064,  0.000
      5: 1.069,  0.063


Table: gen
            pg,     qg
      1: 0.400,  0.300
      2: 1.700,  1.275
      3: 3.245,  3.900
      4: 0.000, -0.108
      5: 4.707, -1.650


The solution make sense intuitively.  The voltage magnitude on bus 3 is pushed to the upper limit, presumably to minimize line losses.  Total generation costs are similar to DC-OPF which seems reasonable.

The solution seems reasonable, but is it optimal?  To test that we need a convex relaxation!  But first a brief digression.

A little secret, the functions `run_dc_opf` and `run_ac_opf` are short hands for a much more general `run_opf` function which supports many variants of optimal power flow problem.  Specifically,
```
run_dc_opf(data, solver) => run_opf(data, DCPPowerModel, solver)
run_dc_opf(data, solver) => run_opf(data, ACPPowerModel, solver)
```

Try the following code with different PowerModels.  Suggested models include `ACPPowerModel`,`ACRPowerModel`,`DCPPowerModel`,`DCPLLPowerModel`,`LPACCPowerModel`, `SOCWRPowerModel`, `QCWRPowerModel`.  A fairly complete list of implemented models is available [here](https://lanl-ansi.github.io/PowerModels.jl/stable/formulation-details/).

In [41]:
result = run_opf(data, ACPPowerModel, nlp_solver)
PowerModels.print_summary(result["solution"])
println()
println("Objective: $(result["objective"])")

Metadata
  baseMVA: 100.000
  per_unit: true

Table Counts
  bus: 5
  gen: 5


Table: bus
            vm,     va
      1: 1.078,  0.049
      2: 1.084, -0.013
      3: 1.100, -0.010
      4: 1.064,  0.000
      5: 1.069,  0.063


Table: gen
            pg,     qg
      1: 0.400,  0.300
      2: 1.700,  1.275
      3: 3.245,  3.900
      4: 0.000, -0.108
      5: 4.707, -1.650

Objective: 17551.890838594758


At this point you might be curious what is PowerModels doing under the hood.  In short, it builds and solves JuMP models.  The [PowerModelsAnnex](https://github.com/lanl-ansi/PowerModelsAnnex.jl/tree/master/src/model) includes some examples of the kind of JuMP models that PowerModels when you call `run_opf`.

Another way to see what PowerModels is doing is to pass your own JuMP model, then you can inspect it later.

In [42]:
using JuMP

m = Model()
result = run_opf(data, DCPPowerModel, nlp_solver, jump_model=m)

println(m)

Min 4000 0_1_pg[4] + 1500 0_1_pg[2] + 3000 0_1_pg[3] + 1000 0_1_pg[5] + 1400 0_1_pg[1]
Subject to
 0_1_pg[4] ≥ 0.0
 0_1_pg[2] ≥ 0.0
 0_1_pg[3] ≥ 0.0
 0_1_pg[5] ≥ 0.0
 0_1_pg[1] ≥ 0.0
 0_1_p[(4, 2, 3)] ≥ -4.26
 0_1_p[(2, 1, 4)] ≥ -4.26
 0_1_p[(3, 1, 5)] ≥ -4.26
 0_1_p[(5, 3, 4)] ≥ -4.26
 0_1_p[(6, 4, 5)] ≥ -2.4
 0_1_p[(1, 1, 2)] ≥ -4.0
 0_1_pg[4] ≤ 2.0
 0_1_pg[2] ≤ 1.7
 0_1_pg[3] ≤ 5.2
 0_1_pg[5] ≤ 6.0
 0_1_pg[1] ≤ 0.4
 0_1_p[(4, 2, 3)] ≤ 4.26
 0_1_p[(2, 1, 4)] ≤ 4.26
 0_1_p[(3, 1, 5)] ≤ 4.26
 0_1_p[(5, 3, 4)] ≤ 4.26
 0_1_p[(6, 4, 5)] ≤ 2.4
 0_1_p[(1, 1, 2)] ≤ 4.0
 0_1_va[4] = 0.0
 0_1_p[(6, 4, 5)] - 0_1_p[(2, 1, 4)] - 0_1_p[(5, 3, 4)] - 0_1_pg[4] = -4.0
 0_1_p[(4, 2, 3)] - 0_1_p[(1, 1, 2)] = -3.0
 0_1_p[(5, 3, 4)] - 0_1_p[(4, 2, 3)] - 0_1_pg[3] = -3.0
 -0_1_p[(3, 1, 5)] - 0_1_p[(6, 4, 5)] - 0_1_pg[5] = 0.0
 0_1_p[(2, 1, 4)] + 0_1_p[(3, 1, 5)] + 0_1_p[(1, 1, 2)] - 0_1_pg[2] - 0_1_pg[1] = 0.0
 0_1_p[(4, 2, 3)] - 91.67583425009167 0_1_va[2] + 91.67583425009167 0_1_va[3] = 0.0
 0_1_p[(2, 1

### Computing Optimality Gaps

Getting back to the question of AC-OPF solution quality.  An easy way to certify AC-OPF solution quality is to use a convex-relaxation to compute a lower bound.  If the "gap" between the solution and the lower bound is small we are happy.

Let build a helper function for computing optimality gaps.

In [43]:
function soc_gap(data_file, nlp_solver)
    data = PowerModels.parse_file(data_file)
    ac_result = run_opf(data, ACPPowerModel, nlp_solver)
    soc_result = run_opf(data, SOCWRPowerModel, nlp_solver)
    
    # it is important to check the solver terminated nominally!
    @assert  ac_result["termination_status"] == OPTIMAL ||  ac_result["termination_status"] == LOCALLY_SOLVED
    @assert soc_result["termination_status"] == OPTIMAL || soc_result["termination_status"] == LOCALLY_SOLVED
    
    # by convention optimality gaps are given as a percentage
    gap = 100*(ac_result["objective"] - soc_result["objective"])/ac_result["objective"]
    
    return (gap=gap, ub_time=ac_result["solve_time"], lb_time=soc_result["solve_time"]) 
end

soc_gap (generic function with 1 method)

In [44]:
soc_gap("data/pglib_opf_case5_pjm.m", nlp_solver)

(gap = 14.540745922427227, ub_time = 0.017488481, lb_time = 0.017096519)

A 14% gap, that's HUGE!  It is not clear if the AC-OPF solutoin is optimal or not.  Clearly this case requires more study.

In the mean time, try the `soc_gap` function on a few more of the network cases included with this tutorial.  Available datasets are `pglib_opf_case14_ieee.m`, `pglib_opf_case30_ieee.m`, `pglib_opf_case73_ieee_rts.m`, `pglib_opf_case118_ieee.m`, `pglib_opf_case200_tamu.m`, `pglib_opf_case500_tamu.m`, `pglib_opf_case1354_pegase.m`, `pglib_opf_case2383wp_k.m` (larger may take a few minutes to run).  The complete PGLib-OPF benchmark library with over 100 cases is available [here](https://github.com/power-grid-lib/pglib-opf).

In [45]:
soc_gap("data/pglib_opf_case14_ieee.m", nlp_solver)

(gap = 0.1090941641707952, ub_time = 0.026690722, lb_time = 0.027486757)

#### Discussion Point

Can you draw any conclusions from computing optimality gaps on a variety of cases?


### A Simple Contingency Study

Component `status` parameters can be useful for exploring a variety of component failure scenarios.  Lets use it to do a simple generator N-1 study on the 5-bus example.


In [46]:
function single_generator_loss(data_file, nlp_solver)
    data = PowerModels.parse_file(data_file)
    
    result = run_opf(data, ACPPowerModel, nlp_solver)

    # it is important to check the solver terminated nominally!
    @assert  result["termination_status"] == OPTIMAL || result["termination_status"] == LOCALLY_SOLVED

    println("base cost: $(result["objective"])")
    
    for (i,gen) in sort(data["gen"])
        # remove the generator
        gen["gen_status"] = 0
        
        result = run_opf(data, ACPPowerModel, nlp_solver)
        
        println("gen $(i) out: $(result["objective"]), $(result["termination_statusstatus"])")

        # restore the generator
        gen["gen_status"] = 1
    end
end

single_generator_loss (generic function with 1 method)

In [47]:
single_generator_loss("data/pglib_opf_case5_pjm.m", nlp_solver)

base cost: 17551.890838594758


KeyError: KeyError: key "termination_statusstatus" not found

This simple study seems to suggest that this network is not N-1 secure and cannot support the loss of generator 3 or 5.  Let's compute how much these two generators contribute to the total generation capacity in the data set.

In [48]:
pg_capacity = sum(gen["pmax"] for (i,gen) in data["gen"])

15.3

In [49]:
100*data["gen"]["3"]["pmax"]/pg_capacity

33.98692810457516

In [50]:
100*data["gen"]["5"]["pmax"]/pg_capacity

39.2156862745098

It seems reasonable that losing over 30% of the total generating capacity would be problematic.

### Solving Optimal Transmission Switching (OTS)

Because InfrastructureModels is built on JuMP, is it fairly easy to encode challenging mixed integer nonlinear optimization problems.  Solving these problems however presents additional challenges.

Lets consider the OTS problem.  An extension of OPF where branches can be removed from the network (i.e. a binary decion).  Lets try solving OPF and OTS on the 5-bus example.  Because the OTS problem has discrete variables we will need to use a MIP solver for that one.

In [51]:
data = PowerModels.parse_file("data/pglib_opf_case5_pjm.m")

opf_result = run_opf(data, DCPPowerModel, lp_solver)

┌ Warning: MathOptInterface.VariablePrimalStart() is not supported by MathOptInterface.Bridges.LazyBridgeOptimizer{GLPK.Optimizer,MathOptInterface.Utilities.UniversalFallback{MathOptInterface.Bridges.AllBridgedConstraints{Float64}}}. This 
│   information will be discarded. = information will be discarded.
└ @ MathOptInterface.Utilities /Users/carleton/.julia/packages/MathOptInterface/C3lip/src/Utilities/copy.jl:133


Dict{String,Any} with 10 entries:
  "solve_time"         => 8.58307e-5
  "optimizer"          => "GLPK"
  "termination_status" => OPTIMAL
  "dual_status"        => FEASIBLE_POINT
  "primal_status"      => FEASIBLE_POINT
  "objective"          => 17479.9
  "solution"           => Dict{String,Any}("baseMVA"=>100.0,"gen"=>Dict{String,…
  "machine"            => Dict("cpu"=>"Intel(R) Core(TM) i7-4870HQ CPU @ 2.50GH…
  "data"               => Dict{String,Any}("name"=>"pglib_opf_case5_pjm","bus_c…
  "objective_lb"       => -Inf

In [55]:
mip_solver = with_optimizer(GLPK.Optimizer)

ots_result = run_ots(data, DCPPowerModel, mip_solver)

┌ Warning: MathOptInterface.VariablePrimalStart() is not supported by MathOptInterface.Bridges.LazyBridgeOptimizer{GLPK.Optimizer,MathOptInterface.Utilities.UniversalFallback{MathOptInterface.Bridges.AllBridgedConstraints{Float64}}}. This 
│   information will be discarded. = information will be discarded.
└ @ MathOptInterface.Utilities /Users/carleton/.julia/packages/MathOptInterface/C3lip/src/Utilities/copy.jl:133


ErrorException: Cannot set bounds because variable is of type: BINARY.

Look at those massive cost savings!!!  Lets look at the solution to see what happened.

In [53]:
PowerModels.print_summary(ots_result["solution"])

UndefVarError: UndefVarError: ots_result not defined

It seems that removing branch 5 from the network will remove congestion caused by Ohm's Law and yield the observed savings.

#### OTS Home Work

Explore the AC-OTS problem on the 5-bus case.  Build the model with commands like `run_ots(data, ACPPowerModel, <solver>)` and `run_ots(data, SOCWRPowerModel, <solver>)`.  Use solvers like Juniper/Bonmin as heuristics to find upper bound solutions and solvers like Juniper/Pavito to find lower bounds.  Compare the gaps similar to how we did for the OPF probem.

### Other Topics

PowerModels includes a variety of useful features that are outside the scope of this brief tutorial.  These features include,
* Extracting Duals/LMPs from linear models using output options
* Network models with HVDC lines or storage devices
* Optimization Based Bound Tightening algorithms
* Using mulitnetworks for time series analysis
* Using multiconductors to model three phase networks (e.g. see [ThreePhasePowerModels](https://github.com/lanl-ansi/ThreePhasePowerModels.jl))
* A variety of experimental features in [PowerModelsAnnex](https://github.com/lanl-ansi/PowerModelsAnnex.jl)